In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from pmdarima import auto_arima

# Import Data

In [12]:
df = pd.read_csv('data/SAP Datasets.csv')

# forward and backward filling
years_col = [str(i) for i in range(2000,2024)]
df[years_col] = df[years_col].apply(lambda row: row.ffill().bfill(), axis=1)

# drop unnecessary columns
df.drop(['Country Code', 'short description', 'long description',], axis=1)

/var/folders/ds/b8dd7ztn06l6p5dfvy90q01c0000gn/T/ipykernel_1408/3371988053.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/SAP Datasets.csv')


,Country Name,Indicator Name,Topic,Indicator Code,Unit of measure,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,Control of Corruption: Estimate,Public Sector: Policy & institutions,CC.EST,0,1.165965,1.165965,1.165965,1.165965,1.165965,...,1.014232,1.248461,1.232735,1.239377,1.207429,1.174703,1.199392,0.801611,0.756785,0.756785
1,Africa Eastern and Southern,Control of Corruption: Estimate,Public Sector: Policy & institutions,CC.EST,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,Control of Corruption: Estimate,Public Sector: Policy & institutions,CC.EST,0,-1.271724,-1.271724,-1.251137,-1.344180,-1.350647,...,-1.364743,-1.354240,-1.540353,-1.531910,-1.502881,-1.419499,-1.493700,-1.152327,-1.183776,-1.183776
3,Africa Western and Central,Control of Corruption: Estimate,Public Sector: Policy & institutions,CC.EST,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,Control of Corruption: Estimate,Public Sector: Policy & institutions,CC.EST,0,-1.197514,-1.197514,-1.155493,-1.335463,-1.377569,...,-1.457798,-1.427323,-1.483337,-1.444333,-1.199251,-1.058418,-0.938673,-0.651610,-0.601941,-0.601941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,Kosovo,Urban population (% of total population),Environment: Density & urbanization,SP.URB.TOTL.IN.ZS,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23137,"Yemen, Rep.",Urban population (% of total population),Environment: Density & urbanization,SP.URB.TOTL.IN.ZS,0,26.267000,26.787000,27.315000,27.849000,28.390000,...,34.165000,34.777000,35.394000,36.016000,36.642000,37.273000,37.908000,38.546000,39.188000,39.831000
23138,South Africa,Urban population (% of total population),Environment: Density & urbanization,SP.URB.TOTL.IN.ZS,0,56.891000,57.368000,57.898000,58.446000,58.993000,...,64.312000,64.828000,65.341000,65.850000,66.355000,66.856000,67.354000,67.847000,68.335000,68.819000
23139,Zambia,Urban population (% of total population),Environment: Density & urbanization,SP.URB.TOTL.IN.ZS,0,34.802000,35.002000,35.475000,35.951000,36.430000,...,41.382000,41.907000,42.438000,42.976000,43.521000,44.072000,44.629000,45.192000,45.761000,46.335000


# Ranking by Indicators

In [16]:
unique_indicators = df['Indicator Name'].unique()

indicator_dfs = {}
year_columns = [str(year) for year in range(2000, 2023)]

for indicator in unique_indicators:
    temp_df = df[df['Indicator Name'] == indicator]

    pivot_df = temp_df.pivot(index='Country Name', columns = 'Indicator Name', values='year_columns')

    indicator_dfs[indicator] = pivot_df

KeyError: 'year_columns'